# Optimisation of LCA models

* Minimise environment impact and/or cost of a network of processes 
* OpenLCA for lifecycle assessment data
* Pyomo for mathematical optimisation
* Jupyter notebooks for prototyping
* Qt GUI for configuration and solution

# OpenLCA

* Java application built by Greendata
* Front-end to life-cycle database of processes and flows
* Ecoinvent system database has ~18000 processes ~7000 flows
* System processes broken down into elementary flows
* Contains methodologies to calculate impact of system
* Cannot feasibly optimise over multiple alternatives

# Pyomo

* An algebraic modelling langugage like GAMS, AIMMS
* Implemented as an object model in Python package `pyomo`
* Eases translation of mathematical optimisation problem to actual implementation
* Optimisation problems are formulated by building concrete or abstract model


# Mola

Python package builds optimisation models using openLCA data

| Module | Description |
| ------ | ----------- |
| `mola.dataimport` | Convert openLCA Ecoinvent system db to sqlite |
| `mola.dataview` | Build SQL to query sqlite db |
| `mola.specificationv5` | Classes each containing abstract Pyomo model |
| `mola.build` | Configure, populate, solve optimisation problem |

## Example

`mola.dataview` is a module of functions to build SQL and return Pandas DataFrame.

In [3]:
import mola.dataview as dv
dbconn = dv.get_sqlite_connection()

In [5]:
dv.get_processes(dbconn, name=['%orange%'])

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE '%orange%'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,527856,760f1858-b0ef-3280-930e-88ebe4645061,"orange production, fresh grade | orange, fresh...",LCI_RESULT,Rest-of-World,527858
1,585243,a334aef7-747b-3ad9-a69a-58ffd56462e6,"market for orange, fresh grade | orange, fresh...",LCI_RESULT,Global,585245
2,3730189,cfc3f58f-7bdf-3019-888b-c4f2dbc332e8,"orange production, fresh grade | orange, fresh...",LCI_RESULT,United States,3730191
3,5834148,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,"orange production, fresh grade | orange, fresh...",LCI_RESULT,South Africa,5834150
4,14457362,637362e8-7c01-3c4a-b3d0-2a8fd12f2954,"orange production, processing grade | orange, ...",LCI_RESULT,United States,14457364
5,14949273,b284de8a-5397-3208-85a0-e3a4148bebac,"market for orange, processing grade | orange, ...",LCI_RESULT,Global,14949275
6,17165213,f2bffdb7-d86c-3c11-b385-f3f18e8afbb1,"orange production, processing grade | orange, ...",LCI_RESULT,Brazil,17165215
7,17667458,a5e1c349-0c5b-3dc6-a04e-6ebee14f5d8c,"orange production, fresh grade | orange, fresh...",LCI_RESULT,Spain,17667460
8,28232925,c29a4516-eaba-391f-b91d-f1828b1d7939,"orange production, processing grade | orange, ...",LCI_RESULT,Rest-of-World,28232927


## Specification

Decorates a `pyomo` abstract model so that we can build an interface using introspection.

```
class TestSpecification(Specification):
    name = 'Test Specification'
    user_defined_sets = {'A': {'doc': 'test set', 'lookup': False}}
    user_defined_parameters = {'Q': {'index': ['A'], 'doc': 'test parameters'}}

    def __init__():
        self.abstract_model = AbstractModel()
        # build user defined and db parameters
        
    def populate(json_files): # populates abstract model
        dp = DataPortal()
        
```

# MolaQT

Front-end to Mola using `pyqt`. The app is built around a `Controller` object for each model

```
class Controller(QWidget):

  def __init__(spec):
      self.spec = spec
      self.sets = spec.get_default_sets()
      self.parameters = spec.get_default_parameters()
      self.lookup = LookupTables(self.db) # if required
  
  def get_config(self): # get current state
```

## StandardController

Aims to Mirror Pyomo modelling language using a collection of Qt Widgets

```
class StandardController(Controller):

  def __init__(spec):
       self.sets_editor = SetsEditor(self.sets, self.lookup)
       self.parameters_editor = ParametersEditor(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
  
```

## CustomController

High-level widgets tied to a particular model and workflow. Hardcodes the model to the UI.

```
class StandardController(Controller):

  def __init__(spec):
       self.objectives = ObjectivesWidget(self.sets, self.lookup)
       self.ProcessFlow = ProcessFlow(self.sets, self.parameters, self.lookup)
       self.model_build = ModelBuild(self)
       self.model_run = ModelRun(self.lookup)
 ```

## DashboardController

* Not yet implemented
* No need to use tabbed interface
* Could create a single dashboard if model is sufficiently simple
* Incorporate or export output

# Demo of molaqt

* General overview: Github, DB, General Specification, Models
* AIMMS tutorial problem: interface and model construction
* Orange production and transport: openLCA data (StandardController)
* Orange production and transport: openLCA data (CustomController)

# Development work

* Tasks: pyomo.network - MultiTaskController, ScheduleController, DashboardController
* Storage: AIMMS advanced example, Sargent papers, STNs, RTNs
* Output: current module to convert pyomo output to DataFrames for export, graphing
* More sophistical openLCA models e.g. oil to diesel conversion
* Dynamic model generation?